# Estimación de la población flotante en Torrevieja

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.express as px
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(0)
pd.set_option("display.max_columns", None)

## Análisis y limpieza de datos

In [ ]:
df = pd.read_csv('/kaggle/input/datos-consumo-agua-torrevieja/AGAMED_DATOS_2017-2018.csv',sep=";")

In [ ]:
df

Eliminamos las filas que contengan valores nulos

In [ ]:
df = df.dropna(how='all')

Convertimos columna Date, a formato datetime.

In [ ]:
df['$Date'] = pd.to_datetime(df['$Date'],format='%d/%m/%Y')



In [ ]:
columnas = df.columns[2:] 

for i in columnas:
    df[i] = df[i].str.replace(',','.')
    df[i] = pd.to_numeric(df[i])

In [ ]:
df_original = df.copy()

In [ ]:
df_original

Agrupamos los valores por fechas, de tal manera que acumulamos por cada columna el valor correspondiente a dicho día.

In [ ]:
df = df.groupby(by='$Date').sum().drop(columns='$Time').round(2).reset_index()

In [ ]:
df

Comparamos el consumo de los contadores de la zona norte.

In [ ]:
# Crear el gráfico de líneas con Plotly
fig = px.line(df, x='$Date', y=[
    'Pozo_Dulce_NORTE',
    'La_Hoya_NORTE',
], labels={
    'variable': 'Tipo de Consumo',
    'value': 'Consumo de Agua',
    '$Date': 'Date'
}, title='Comparación de Contadores Zona Norte')

# Configurar los nombres de las líneas
fig.for_each_trace(lambda t: t.update(name={
    'Pozo_Dulce_NORTE': 'Consumo Contador Pozo Dulce Norte',
    'La_Hoya_NORTE': 'Consumo Contador la hoya Norte',
}[t.name]))

# Mostrar el gráfico
fig.show()


Comparamos el consumo de los contadores de la zona sur.

In [ ]:
# Crear el gráfico de líneas con Plotly
fig = px.line(df, x='$Date', y=[
    'contador Ent. Taibilla III_SUR',
    'Cont llenado la Hoya_SUR',
    'Contador Avda. Diego Ramírez_SUR',
    'Contador Avda. Diego Ramírez_SUR.1'
    
], labels={
    'variable': 'Tipo de Consumo',
    'value': 'Consumo de Agua',
    '$Date': 'Date'
}, title='Comparación de Contadores Zona Sur')

# Configurar los nombres de las líneas
fig.for_each_trace(lambda t: t.update(name={
    'contador Ent. Taibilla III_SUR': 'Consumo Contador Ent. Taibilla III_SUR',
    'Cont llenado la Hoya_SUR': 'Consumo Contador la Hoya_SUR',
    'Contador Avda. Diego Ramírez_SUR': 'Consumo Contador Avda. Diego Ramírez_SUR',
    'Contador Avda. Diego Ramírez_SUR.1': 'Contador Avda. Diego Ramírez_SUR. (1)'
}[t.name]))

# Mostrar el gráfico
fig.show()


In [ ]:
df = df.sort_values(by='$Date',ascending=True)

Analizamos el consumo total acumulado en los contadores de las diferentes zonas.

In [ ]:
df['TOTAL_SUR'] = df['contador Ent. Taibilla III_SUR'] + df['Cont llenado la Hoya_SUR'] + df['Contador Avda. Diego Ramírez_SUR'] + df['Contador Avda. Diego Ramírez_SUR.1']
df['TOTAL_NORTE'] = df['Pozo_Dulce_NORTE']+df['La_Hoya_NORTE']
df['TOTAL_TRV'] = df['TOTAL_NORTE'] + df['TOTAL_SUR']

In [ ]:
df

In [ ]:
# Crear el gráfico de líneas con Plotly
fig = px.line(df, x='$Date', y=[
    'TOTAL_SUR',
    'TOTAL_NORTE',
], labels={
    'variable': 'Tipo de Consumo',
    'value': 'Consumo de Agua',
    '$Date': 'Date'
}, title='Comparación de Consumo Total')

# Configurar los nombres de las líneas
fig.for_each_trace(lambda t: t.update(name={
    'TOTAL_SUR': 'Consumo total zona Sur',
    'TOTAL_NORTE': 'Consumo total zona Norte',
}[t.name]))

# Mostrar el gráfico
fig.show()


En todas las gráficas podemos observar que los valores máximos se suelen alcanzar en período vacacional de verano(Junio-Septiempre), por lo general en ambas zonas el consumo sigue una evolución similar. Cabe recalcar que al disponer de mayor cantidad de contadores de la zona sur, el valor acumulado es mayor en comparación con la zona norte.

## Análisis de los nuevos conjuntos de datos 

In [ ]:
caudales_diarios_original = pd.read_excel("/kaggle/input/datos-consumo-agua-torrevieja/ESTUDIO ESTIMACION POBLACION FLOTANTE EN TORREVIEJA_DATOS AGAMED.xlsx", sheet_name="CAUDALES DIARIOS")
zona_tt3_original = pd.read_excel("/kaggle/input/datos-consumo-agua-torrevieja/ESTUDIO ESTIMACION POBLACION FLOTANTE EN TORREVIEJA_DATOS AGAMED.xlsx", sheet_name="TLL zona T33", skiprows=1)
zona_tt4_original =pd.read_excel("/kaggle/input/datos-consumo-agua-torrevieja/ESTUDIO ESTIMACION POBLACION FLOTANTE EN TORREVIEJA_DATOS AGAMED.xlsx",sheet_name="TLL zona T34", skiprows=1)


A continuación, describimos cada una de las columnas correspondientes a la hoja 1 del excel, denominada `CAUDALES DIARIOS`, en ella se definen estos campos.

* **FECHA**: Fecha de los datos registrados.
* **TOTAL TORREVIEJA**: Consumo Total en m³ en la ciudad de Torrevieja.
* **SUBTOTAL CASCO URBANO (EN AQVD SUBSECTOR CASCO URBANO)**: Consumo en m³ en el casco urbano de Torrevieja.
* **SUBTOTAL ROCIO DEL MAR (EN AQVD SUBSECTOR R. DEL MAR)**: Consumo en m³ en la zona de Rocío del Mar.
* **SUBTOTAL LA MATA (EN AQVD SUBSECTOR RESTO LA MATA)**: Consumo en m³ en la zona de La Mata.
* **CAUDAL SALIDA EDAR (m³)**: Caudal de salida de la estación depuradora de aguas residuales (EDAR), medido en metros cúbicos.
* **DBO (mg/l)**: Demanda Biológica de Oxígeno, medida en miligramos por litro.
* **DQO (mg/l)**: Demanda Química de Oxígeno, medida en miligramos por litro.
* **N-NH4 (mg/l)**: Concentración de amonio (NH4), medida en miligramos por litro.
* **N-NO3 (mg/l)**: Concentración de nitrato (NO3), medida en miligramos por litro.
* **N total (mg/l)**: Concentración total de nitrógeno, medida en miligramos por litro.
* **Habitantes Eq.**: Habitantes equivalentes, que representan la carga orgánica generada por la población.
* **Habitantes Eq. Semanal**: Habitantes equivalentes calculados semanalmente.
* **Habitantes Eq. RD 11/1995**: Habitantes equivalentes según la normativa RD 11/1995.

In [ ]:
caudales_diarios_original.sample(3)

Para las hojas restantes `TLL zona T33` y `TLL zona T34`, recopilan información específica del consumo de agua por litros por zona.

* **Subsector**: Área específica a la que se corresponde el dato, los valores pueden ser [Zona T33, Zona T34]
* **Fecha**: Fecha en la que se registró el consumo.
* **Consumo (litros)**: Cantidad de agua consumida, medida en litros.
* **Tipo de consumo**: Categoría o clasificación del consumo de agua, en todas las muestras que se proporcionan tienen el valor asociado *Real*.



In [ ]:
zona_tt3 = zona_tt3_original.copy()
zona_tt3['Fecha'] = pd.to_datetime(zona_tt3['Fecha']).dt.date
zona_tt3.sample(3)

In [ ]:
zona_tt4 = zona_tt4_original.copy()
zona_tt4.columns = ['Subsector', 'Fecha', 'Consumo (litros)', 'Tipo de consumo']
zona_tt4['Fecha'] = pd.to_datetime(zona_tt4['Fecha']).dt.date
zona_tt4.sample(3)

In [ ]:
consumo_zona_fecha = pd.concat([zona_tt3, zona_tt4])
consumo_zona_fecha.sample(5)

In [ ]:
consumo_zona_fecha.nunique()

In [ ]:
# Eliminamos tipo de consumo ya que es irrelevante, son todos iguales
del consumo_zona_fecha['Tipo de consumo']
consumo_zona_fecha.sort_values(by=['Fecha'],ascending=True, inplace=True)
consumo_zona_fecha


En primer lugar, analizamos la distribución de la columna **Subsector** para observar valores erróneos, es decir, si existen valores nulos o bien que sean negativos.

In [ ]:
consumo_zona_fecha.info()

Analizando los valores asociados al subsector Zona T34, observamos lo siguiente.
* Valor mínimo negativo
* Valores atípicos.


Para solucionar este problema, vamos a aplicar diferentes técnicas de imputación de datos para la correcta construcción de los datos que nos permita un análisis de calidad y representativo.

In [ ]:
zona_tt4.describe()

Ponemos los valores negativos a 0.

In [ ]:
zona_tt4['Consumo (litros)'][zona_tt4['Consumo (litros)'] <  0] = 0

In [ ]:
zona_tt4[zona_tt4['Consumo (litros)'] ==  0]

Analizando los valores atípicos, observamos que tenemos un valor que se encuentra muy alejado de los valores usuales de la muestra para esta zona en concreto, para ello y con el objetivo de mejorar el ajuste, aplicaremos el rango intercuartílico para eliminar dichos valores.

In [ ]:
plt.figure(figsize=(10, 6))
zona_tt4[['Consumo (litros)']].boxplot(flierprops={'marker':'o', 'markersize':10, 'markeredgecolor':'red'})
plt.title('Boxplot de la zona T34')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = zona_tt4['Consumo (litros)'].quantile(0.25)
Q3 = zona_tt4['Consumo (litros)'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier criteria
threshold = 1.5
outlier_condition = ((zona_tt4['Consumo (litros)'] < (Q1 - threshold * IQR)) | (zona_tt4['Consumo (litros)'] > (Q3 + threshold * IQR)))

# Filter out outliers
zona_tt4 = zona_tt4[~outlier_condition]

zona_tt4

Tras eliminar los valores atípicos,la distribución se quedaría de la siguient manera.

In [ ]:
plt.figure(figsize=(10, 6))
zona_tt4[['Consumo (litros)']].boxplot(flierprops={'marker':'o', 'markersize':10, 'markeredgecolor':'red'})
plt.title('Boxplot de la zona T34')
plt.xticks(rotation=45)
plt.show()

Para aquellos valores que hemos puesto a 0, para que el valor asignado no afecte a la media de la muestra, vamos a asignarles a dichas huecos el valor de la media sin tener en cuenta estos valores.

In [ ]:
zona_tt3.describe()

In [ ]:
zona_tt3['Consumo (litros)'][zona_tt3['Consumo (litros)'] <  0] = 0

In [ ]:
plt.figure(figsize=(10, 6))
zona_tt3[['Consumo (litros)']].boxplot(flierprops={'marker':'o', 'markersize':10, 'markeredgecolor':'red'})
plt.title('Boxplot de la zona T33')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = zona_tt3['Consumo (litros)'].quantile(0.25)
Q3 = zona_tt3['Consumo (litros)'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier criteria
threshold = 1.5
outlier_condition = ((zona_tt3['Consumo (litros)'] < (Q1 - threshold * IQR)) | (zona_tt3['Consumo (litros)'] > (Q3 + threshold * IQR)))

# Filter out outliers
zona_tt3 = zona_tt3[~outlier_condition]

zona_tt3

In [ ]:
plt.figure(figsize=(10, 6))
zona_tt3[['Consumo (litros)']].boxplot(flierprops={'marker':'o', 'markersize':10, 'markeredgecolor':'red'})
plt.title('Boxplot de la zona T33')
plt.xticks(rotation=45)
plt.show()

## Predicción de la población flotante

Los conjuntos de datos que se nos han proporcionado no corresponden al mismo año, por lo que se realizará una aproximación de estos. Para ello se utilizarán los siguientes años:
- Datos de caudales: año 2019
- Datos de la zona T33: año 2021
- Datos de la zona T34: año 2021

In [ ]:
(df.head(3))

In [ ]:
caudales_diarios_original['FECHA'] = pd.to_datetime(caudales_diarios_original['FECHA'])

caudales_diarios_original = caudales_diarios_original[caudales_diarios_original['FECHA'].dt.year == 2019]

(caudales_diarios_original.head(3))

Tras filtrar el año correspondiente se eliminan las columnas que no se utilizarán en la estimación.

In [ ]:
caudales_diarios_original.drop(columns=['DBO\n (mg/l)', 'DQO\n (mg/l)', 'N-NH4\n(mg/l)', 'N-NO3\n(mg/l)', 'N total\n(mg/l)'], inplace=True)

caudales_diarios_original

In [ ]:
zona_tt3['Fecha'] = pd.to_datetime(zona_tt3['Fecha'])

zona_tt3 = zona_tt3[zona_tt3['Fecha'].dt.year == 2021]

(zona_tt3.head(3))



In [ ]:
zona_tt4['Fecha'] = pd.to_datetime(zona_tt4['Fecha'])

zona_tt4 = zona_tt4[zona_tt4['Fecha'].dt.year == 2021]

(zona_tt4.head(3))

### Estimación de la población flotante en la ciudad de Torrevieja

El primer paso para la estimación de la población flotante es la limpieza de los valores atípicos en el conjunto de datos. Para ello, se aplica la estrategia del rango intercuartílico en la columna 'TOTAL_TRV'.

In [ ]:
# eliminar valores atipicos en df
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['TOTAL_TRV'].quantile(0.25)
Q3 = df['TOTAL_TRV'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier criteria
threshold = 1.5
outlier_condition = ((df['TOTAL_TRV'] < (Q1 - threshold * IQR)) | (df['TOTAL_TRV'] > (Q3 + threshold * IQR)))

# Filter out outliers
df = df[~outlier_condition]

df.describe()

En este punto ya no se disponen de valores atipicos

La estrategia para predecir la poblacion flotante se describe a continuación
- Obtener una estimación del promedio de consumo de agua de la ciudad (columna TOTAL_TRV).
- Establecer una relación entre el promedio del consumo y la población censada en el año 2018.
- Calcular a partir de esa relación la población flotante.

Para la obtención de los datos de consumo se realizará un calculo sobre la columna del dataframe, mientras que para la obtención de la población se aplicará la tecnica de Web Scrapping a la pagina web de la Diputación de Alicante, en concreto el enlace web es el siguiente: http://documentacion.diputacionalicante.es/censo.asp?tfm_order=DESC&tfm_orderby=Pobl2018

In [ ]:
df.head(3)

In [ ]:
# Obtención del promedio de consumo de agua en la ciudad

promedio_consumo = df['TOTAL_TRV'].mean()
print(promedio_consumo)

In [ ]:
# Obtención del censo de la ciudad de torrevieja en el año 2018 mediante Web Scrapping

import requests
from bs4 import BeautifulSoup

url = 'http://documentacion.diputacionalicante.es/censo.asp?tfm_order=DESC&tfm_orderby=Pobl2018'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Busqueda de la tabla por su clase
    table = soup.find('table', class_='tabla tabla_ancho')

    if table:
        # Obtener la cuarta fila (Torrevieja)
        fourth_row = table.find_all('tr')[3]

        row_data = []
        cells = fourth_row.find_all('td')
        row_data = [cell.text.strip() for cell in cells]

        df_pobl = pd.DataFrame([row_data])

        #print(df_pobl)

    else:
        print('Error.')
else:
    print(f'Error: {response.status_code}')

# Censo del año 2018
poblacion_2018 = df_pobl.iloc[0, 6]
poblacion_2018 = poblacion_2018.replace('.', '')
poblacion_2018 = int(poblacion_2018)
print(poblacion_2018)


In [ ]:
# Caluclo de la población flotante

consumo = df['TOTAL_TRV']

poblacion = consumo * poblacion_2018 / promedio_consumo

Una vez calculada la relación, se muestra un gráfico con la evolución de la población flotante.

In [ ]:
fig = px.line(x=df['$Date'], y=poblacion, title='Evolución de la Población')
fig.show()


En el gráfico se observa cómo la población de la ciudad aumenta en verano, alcanzando aproximadamente 140 mil personas, mientras que en los meses fríos disminuye.

### Comparación con la estimación proporcionada

A continuación se realizará una comparación con la estimació de la población que se nos ha ofrecido, para ello primero se va a aproximar el año de los datos de los caudales al año del estudio.

In [ ]:
caudales_diarios_original = caudales_diarios_original.dropna(subset=['Habitantes Eq.'])

caudales_diarios_original['FECHA'] = pd.to_datetime(caudales_diarios_original['FECHA'])

# Cambio las fechas del año 2019 para que parezcan del año 2018
caudales_diarios_original.loc[caudales_diarios_original['FECHA'].dt.year == 2019, 'FECHA'] -= pd.offsets.DateOffset(years=1)

caudales_diarios_original

In [ ]:
import plotly.express as px

fig = px.line(x=df['$Date'], y=poblacion, title='Evolución de la Población')
fig.add_scatter(x=caudales_diarios_original['FECHA'], y=caudales_diarios_original['Habitantes Eq.'], mode='lines', name='Estimación proporcionada')
fig.show()


Esta comparación muestra la estimación de la población flotante calculada por nosotros (línea azul) y la estimación proporcionada en el conjunto de datos (línea roja).

Se observa que la estimación proporcionada no es lineal y estima más del doble de la población en comparación con nuestra estimación.

### Trabajo adicional - Predicción de la población en la Mata (zona T33)

Como trabajo adicional, hemos realizado una predicción en la población flotante de los habitantes de la Mata, siguiendo la misma estrategia que para la ciudad de Torrevieja.

Para ello el primer paso es el de juntar ambas zonas de las que obtienen datos de consumo del agua, seguidamente se aplica la misma estrategia. La población del censo de la Mata se ha obtenido de la siguiente pagina web: https://www.foro-ciudad.com/alicante/torrelamata/habitantes.html, en concreto 1.198 habitantes.

In [ ]:
zona_tt3 = zona_tt3.drop(columns=['Subsector', 'Tipo de consumo'])

In [ ]:
zona_tt4 = zona_tt4.drop(columns=['Subsector', 'Tipo de consumo'])

In [ ]:
la_mata = pd.merge(zona_tt3, zona_tt4, on='Fecha', how='inner')

la_mata['Consumo_total'] = la_mata['Consumo (litros)_x'] + la_mata['Consumo (litros)_y']

la_mata.drop(['Consumo (litros)_x', 'Consumo (litros)_y'], axis=1, inplace=True)

(la_mata)


In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = la_mata['Consumo_total'].quantile(0.25)
Q3 = la_mata['Consumo_total'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier criteria
threshold = 1.5
outlier_condition = ((la_mata['Consumo_total'] < (Q1 - threshold * IQR)) | (la_mata['Consumo_total'] > (Q3 + threshold * IQR)))

# Filter out outliers
la_mata = la_mata[~outlier_condition]

la_mata

In [ ]:
la_mata.describe()

In [ ]:
consumo = la_mata['Consumo_total']

poblacion = consumo * 1198 / 109918.026059

fig = px.line(x=la_mata['Fecha'], y=poblacion, title='Evolución de la Población')
fig.show()

# Problemas encontrados durante la realización de la practica

Al inicio del desarrollo de la practica nos hemos encontrado con diferentes problemas que han imposibilitado la realización de una predicción de la población flotante exacta.

Al inicio del desarrollo de la práctica, nos enfrentamos a una serie de desafíos que obstaculizaron nuestra capacidad para realizar una predicción exacta de la población flotante de Torrevieja. 

Los problemas mencionados a continuación destacan la complejidad de la tarea de prever la población flotante y subrayan la importancia de abordar y superar estos obstáculos para lograr resultados precisos y significativos.

## Desconocimiento de los datos ofrecidos

Entre estos desafíos, se destaca principalmente el desconocimiento de los datos ofrecidos por el profesorado.

Los datos proporcionados abarcan el período comprendido entre los años 2017 y 2018 e incluyen lecturas de contadores de agua de diversas zonas de la ciudad de Torrevieja, tanto en la región sur como en la norte. 

En primer lugar, no disponemos de información clara sobre si se trata de datos de agua consumida o de agua suministrada. Esta distinción es fundamental para comprender adecuadamente los patrones de consumo y/o la distribución de los recursos en la ciudad. 

Además, la falta de especificación sobre la unidad de medida de los datos añade otra capa de complejidad a nuestra comprensión de los mismos, ya que esta información es crucial para realizar análisis precisos y comparaciones significativas.

## Inexistencia de datos de consumo de agua

El siguiente problema tiene que ver con la inexistencia de datos sobre el consumo de agua.

Este aspecto es fundamental para entender los hábitos y comportamientos de la población en relación con el uso del agua, lo cual tiene un impacto directo en la movilidad de la población flotante

Hasta el momento, hemos logrado obtener únicamente información sobre la cantidad de abastecimiento proporcionada por el Ayuntamiento de Torrevieja, expresada en términos de litros por habitante por día, con un valor registrado de 120.13 litros por habitante por día. 

Aunque este dato ofrece una perspectiva general sobre el suministro de agua en la región, la falta de detalles específicos sobre el consumo real limita nuestra capacidad para realizar un análisis completo y preciso de los patrones de movilidad asociados al consumo de agua.

Sería beneficioso obtener información adicional sobre el consumo de agua, especialmente si se desglosa por mes, por ejemplo. Esto nos permitiría capturar variaciones estacionales y tendencias a lo largo del año, lo que resultaría muy útil para comprender mejor los patrones de consumo de agua y su relación con la movilidad de la población.

Este dato se pueden consultar en la web del SINAC (Sistema de Información Nacional de Agua de Consumo) a partir del siguiente enlace https://sinac.sanidad.gob.es/CiudadanoWeb/ciudadano/informacionAbastecimientoActionDetalleRed.do

## Indisponibilidad de datos (secreto estadístico)

Otro obstáculo significativo fue la indisponibilidad de ciertos datos debido a restricciones de confidencialidad estadística, lo que limitó nuestra capacidad para acceder a información valiosa para el análisis.

En el portal del INE (Instituto Nacional de Estadística), se encuentran disponibles datos sobre viajeros y pernoctaciones en diversos puntos turísticos. Esta información habría sido sumamente valiosa para complementar los datos de ciudadanos censados en Torrevieja, permitiéndonos realizar predicciones más precisas sobre la población flotante.

Lamentablemente, nos enfrentamos a una limitación significativa: el acceso a estos datos está restringido y se anuncia como "protegido por secreto estadístico". Esta restricción impide que podamos utilizar esta fuente de información para mejorar nuestro análisis y nuestras predicciones.

El enlace para acceder al conjunto de datos del INE es el siguiente https://ine.es/jaxiT3/Tabla.htm?t=2078&L=0

## Inexistencia de datos de turismo útiles

Por último, la falta de datos de turismo útiles también representó un desafío, ya que estos datos, al ser Torrevieja una ciudad muy visitada por turistas, podrían haber proporcionado información importante sobre los flujos de personas en ella en diferentes períodos de tiempo. 

En la página web de Dataestur, hemos identificado varios conjuntos de datos relacionados con el turismo que podrían haber sido extremadamente útiles para nuestro análisis. Sin embargo, algunos de estos conjuntos de datos no estaban desglosados por municipios ni incluían información para los años específicos de nuestro estudio.

Por ejemplo esta disponible un conjunto de datos titulado ‘Turismo a partir de telefonía móvil’ en el que se muestra la evolución del número de turistas lo largo del año en diferentes municipios, pero en este caso no se dispone de los años para los cuales tenemos que realizar el estudio.

Asimismo, en este portal también encontramos datos sobre las pernoctaciones hoteleras en varios destinos turísticos. Aunque disponemos de información para los años de interés, no se proporciona un desglose de estas pernoctaciones por municipios específicos.

Los diferentes conjuntos de datos se pueden consultar en https://www.dataestur.es